In [ ]:
!pip install google-cloud-bigquery google-auth db-dtypes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import pandas as pd

In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('./multicampus_project_first_account.json')

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project = credentials.project_id
)

===============================================================================================================================================================================================================================

In [ ]:
# 1번 : 자동차 등록 수 자료
# 기준 : 2023년 10월 31일

sql = '''
SELECT
  Year,
  Total
FROM
  project_1.Registration_All
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
sns.barplot(
    data = df,
    x = 'Year',
    y = 'Total'
    ).set(
        title = '자동차 등록대수'
        )
plt.xlabel('연도')   # https://wikidocs.net/92081  x축,y축 레이블 설정
plt.ylabel('')


In [ ]:
# 상승률 계산


answer = []
num = df['Total']

for i in range(len(df['Total'])-1):
    answer.append(round((num[i+1]-num[i])/num[i]*100,2))
print(answer)
print(sum(answer)/len(answer))

In [ ]:
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
plt.title('자동차 등록대수 연평균 변화율')
plt.plot(years,answer)
plt.xlim(2012,2024)
plt.ylim(0,5)
plt.show()

# 그래프 그리기 : https://wikidocs.net/92082

===============================================================================================================================================================================================================================

In [ ]:
# 2번 : 전기차 등록 수 자료
# 기준 : 2023년 10월 31일

sql = '''
SELECT
  Year,
  Total
FROM
  project_1.Registration_EV
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
plt.xticks(rotation=45)
sns.barplot(
    data = df,
    x = 'Year',
    y = 'Total'
    ).set(
        title = '전기차 등록대수'
        )
plt.xlabel('연도')
plt.ylabel('')

In [ ]:
# 상승률 계산


answer = []
num = df['Total']

for i in range(len(df['Total'])-1):
    answer.append(round((num[i+1]-num[i])/num[i]*100,2))
print(answer)
print(sum(answer)/len(answer))

In [ ]:
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
plt.title('전기차 등록대수 연평균 상승률')
plt.plot(years,answer)
plt.xlim(2012,2024)
plt.ylim(0,300)
plt.show()

===============================================================================================================================================================================================================================

In [ ]:
# 3  번 : 전체 충전소 vs 고속도로 충전소 수 비교
# 기준 : 2023년 11월 14일[Text(0.5, 1.0, 'The Number of Registered EV')]


sql = '''
SELECT
  COUNT(Facility_2)
FROM
  project_1.ChargingStations
WHERE
  Facility_2 LIKE '%휴게소%'
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
sql = '''
SELECT
  COUNT(Facility_2)
FROM
  project_1.ChargingStations
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
y = ['총합','고속도로']
x = [265457,1390]

# sns.set_style('whitegrid')
plt.figure(figsize=(8, 5))   # https://rfriend.tistory.com/710
bar = sns.barplot(x=x,y=y)
plt.title('충전소 등록대수')


In [ ]:
ratio = [99.48,0.52]
labels = ['총합','고속도로']
colors = ['skyblue','green']

plt.pie(ratio,labels = labels, autopct = '%.1f%%',startangle = 130,colors=colors)
plt.title('전기차 충전소 등록대수')
plt.show()

# 파이차트 그리기 https://wikidocs.net/92114

===============================================================================================================================================================================================================================

In [ ]:
# 4 - 1 번 : 고속도로 평균 충전량
#timestamp_diff 구하기 https://dev-jwblog.tistory.com/137

sql = '''
SELECT
  COUNT(Type) AS number,
  Type,
  AVG(Amount) AS amount,
  AVG(TIMESTAMP_DIFF(End_Time,Start_Time,MINUTE)) AS time
FROM
  `apt-rite-404708.project_1.ChargingData_2023*`
WHERE
  Station_Name LIKE '%휴게소%'
GROUP BY
  Type
'''

In [ ]:
df = client.query(sql).to_dataframe()
df.columns = ['충전기수', '타입', '충전량', '충전시간']
df

# https://mizykk.tistory.com/131 데이터 열 이름 변경

In [ ]:
# 4 - 2 번 : 고속도로 충전량 

sql = '''
SELECT
  Type,
  Amount
FROM
  `apt-rite-404708.project_1.ChargingData_2023*`
WHERE
 Station_Name LIKE '%휴게소%'
'''

df = client.query(sql).to_dataframe()
df

In [ ]:
plt.figure(figsize=(13, 7))
sns.histplot(
    data = df,
    x = 'Amount',
    bins=[x for x in range(0,100,2)]
    ).set(
        title = '고속도로 전기차 충전량'
    )

In [ ]:
# 4 - 3 번 : 고속도로 충전시간

sql = '''
SELECT
  Type,
  TIMESTAMP_DIFF(End_Time,Start_Time,MINUTE) AS time
FROM
  `apt-rite-404708.project_1.ChargingData_2023*`
WHERE
 Station_Name LIKE '%휴게소%'
'''

df = client.query(sql).to_dataframe()
df

In [ ]:
plt.figure(figsize=(13, 7))
sns.histplot(
    data = df,
    x = 'time',
    bins=[x for x in range(0,100,2)]
    ).set(
        title = '고속도로 전기차 충전시간'
    )

In [ ]:
# 고속도로 노선 수 
sql = '''
SELECT
  COUNT(Route_Name),
  Route_Name
FROM
  project_1.Traffic_2023_11_12
GROUP BY
  Route_Name
'''

df = client.query(sql).to_dataframe()
df

===============================================================================================================================================================================================================================

In [ ]:
# 5번 : 고속도로 통행량

sql = '''
SELECT
  Route_Name,
  SUM(Traffic_Volume) AS traffic
FROM
  project_1.Traffic_2023_11_12
GROUP BY
  Route_Name
ORDER BY
  traffic DESC
LIMIT
 15
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(
    data = df,
    x = 'traffic',
    y = 'Route_Name'
).set(
    
    title = '고속도로 교통량 상위 15'
)
plt.xlabel('교통량')
plt.ylabel('노선명')

===============================================================================================================================================================================================================================

In [ ]:
# 6번 : 고속도로 통행량
# 기준 : 2022년 일평균


sql = '''
SELECT
  Section,
  SUM(DailyAverage) AS DA
FROM
  project_1.Traffic_2022
GROUP BY
  Section
ORDER BY
  SUM(DailyAverage) DESC
LIMIT
  15
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(
    data=df,
    y='Section',
    x='DA'
).set(
    title = '고속도로 구간별 일일 교통량 상위'
)
plt.xlabel('교통량')
plt.ylabel('고속도로 구간')

===============================================================================================================================================================================================================================

이건 좀 아니잖아...............
그래프 다 해봤는데 너무많아서 다 이상함
결론 : 못씀
그럼 우짜라고?

In [ ]:
sql = '''
SELECT
  COUNT(*)
FROM
 project_1.Traffic_RestArea
'''

df = client.query(sql).to_dataframe()
df

In [ ]:
# 7번 : 휴게소 이용객
# 기준 : 23년 8월 24일

sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
  Daily_Visitors IS NOT NULL
ORDER BY
  Daily_Visitors DESC
LIMIT
  15
'''

In [ ]:
df = client.query(sql).to_dataframe()
df

In [ ]:
barplot = sns.barplot(
    data = df,
    y = 'Rest_Area_Name',
    x = 'Daily_Visitors'
).set(
    
    title = '휴게소 방문객 상위 15'
)

plt.xlabel('방문객수')
plt.ylabel('휴게소명')

barplot

===============================================================================================================================================================================================================================

결론도출 ( 함 해보고 )

고속도로 노선 교통량 상위 10개 -> 그중 노선 상위 10개 -> 그중 휴게소 방문대수 상위 10개

===============================================================================================================================================================================================================================

In [ ]:
# 1 : 고속도로 노선 교통량 상위 5개
sql = '''
SELECT
  Route_Name,
  SUM(Traffic_Volume) AS traffic
FROM
  project_1.Traffic_2023_11_12
GROUP BY
  Route_Name
ORDER BY
  traffic DESC
LIMIT
 5
'''
Highway = client.query(sql).to_dataframe()
Highway

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('경부선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_1 = client.query(sql).to_dataframe()
Route_1

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('영동선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_2 = client.query(sql).to_dataframe()
Route_2

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('중부선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_3 = client.query(sql).to_dataframe()
Route_3

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('남해선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_4 = client.query(sql).to_dataframe()
Route_4

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('서해안선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_5 = client.query(sql).to_dataframe()
Route_5

In [ ]:
Ranking = pd.DataFrame(columns = Highway['Route_Name'])
Ranking['경부선'] = Route_1['Rest_Area_Name']
Ranking['영동선'] = Route_2['Rest_Area_Name']
Ranking['중부선-대전통영선A'] = Route_3['Rest_Area_Name']
Ranking['남해선A'] = Route_4['Rest_Area_Name']
Ranking['서해안선'] = Route_5['Rest_Area_Name']

Ranking

#  https://zephyrus1111.tistory.com/33 판다스 표 만들기

끝